In [1]:
import os
import fnmatch
import pandas as pd
import csv
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import matplotlib.pyplot as plt
import pingouin as pg

pd.set_option('display.max_columns', 50)

In [2]:
# Definitions

analysis_path = os.path.dirname(os.getcwd())
path = os.path.dirname(analysis_path)

df_1 = pd.read_excel(f'{analysis_path}/02_Results_concentration_brain.xlsx', header=0)
df_2 = pd.read_excel(f'{analysis_path}/02_Results_exp_2_concentration_brain.xlsx', header=0)

In [3]:
df_1_short = df_1[((df_1['Treatment_Group']=='Har + DMT')) & ((df_1['Component Name']=='DMT'))]
df_2_short = df_2[((df_2['Treatment_Group']=='Har + DMT')) & (df_2['Component Name']=='DMT') & ((df_2['Sample']=='Front Cort') | (df_2['Sample']=='Cerebellum'))]

In [4]:
df_1_front = df_1_short[df_1_short['Sample']=='Front Cort']
df_1_cere = df_1_short[df_1_short['Sample']=='Cerebellum']

df_2_front = df_2_short[df_2_short['Sample']=='Front Cort']
df_2_cere = df_2_short[df_2_short['Sample']=='Cerebellum']


In [5]:
df_2_front

,Sample Name,Treatment_Group_#,Treatment_Group,Sample Number,Dilution Factor,Component Name,Calculated Concentration [ng/g],Sample,mol_weight,molar_concentration nmol/L
49,Rat5_Vial09,2,Har + DMT,1,2.20,DMT,2570.039,Front Cort,188.274,13650.525298
61,Rat6_Vial11,2,Har + DMT,1,2.07,DMT,2163.154,Front Cort,188.274,11489.393119
73,Rat7_Vial13,2,Har + DMT,1,2.70,DMT,1878.217,Front Cort,188.274,9975.976502
85,Rat8_Vial15,2,Har + DMT,1,2.66,DMT,2482.147,Front Cort,188.274,13183.695040
97,Rat9_Vial17,2,Har + DMT,1,2.65,DMT,1519.974,Front Cort,188.274,8073.201823


In [6]:
# normality test

print('Exp 1')
print('Front. Cort')
print(stats.shapiro(df_1_front['Calculated Concentration µg/L']))
print('Cerebellum')
print(stats.shapiro(df_1_cere['Calculated Concentration µg/L']))
print('Exp 2')
print('Front. Cort')
print(stats.shapiro(df_2_front['Calculated Concentration [ng/g]']))
print('Cerebellum')
print(stats.shapiro(df_2_cere['Calculated Concentration [ng/g]']))


Exp 1
Front. Cort
ShapiroResult(statistic=0.8722066879272461, pvalue=0.2351434975862503)
Cerebellum
ShapiroResult(statistic=0.8696577548980713, pvalue=0.22484445571899414)
Exp 2
Front. Cort
ShapiroResult(statistic=0.9440222382545471, pvalue=0.6944968700408936)
Cerebellum
ShapiroResult(statistic=0.9539803266525269, pvalue=0.7655690312385559)


In [7]:
## test for homogeneity of variance between the two groups of each experiment

# Exp 1
stat, p_value = stats.levene(df_1_front['Calculated Concentration µg/L'], df_1_cere['Calculated Concentration µg/L'])

print('Experiment 1')
print(f"Levene's test statistic: {stat}")
print(f"P-value: {p_value}")

# Exp 1
stat, p_value = stats.levene(df_2_front['Calculated Concentration [ng/g]'], df_2_cere['Calculated Concentration [ng/g]'])

print('Experiment 2')
print(f"Levene's test statistic: {stat}")
print(f"P-value: {p_value}")


Experiment 1
Levene's test statistic: 1.9294949140969464
P-value: 0.19496829684625122
Experiment 2
Levene's test statistic: 0.1708982466609166
P-value: 0.6901739145001002


#### -> homogeneity is okay, so t test can be used

In [8]:
### CHeck for differences in DMT distribution between cortex and cerebellum for both experiments
## with paired t-test
# Exp 1
stat, p_value = stats.ttest_rel(df_1_front['Calculated Concentration µg/L'], df_1_cere['Calculated Concentration µg/L'])

print('Experiment 1')
print(f"t-test statistic: {stat}")
print(f"P-value: {p_value}")

# Exp 2 
stat, p_value = stats.ttest_rel(df_2_front['Calculated Concentration [ng/g]'], df_2_cere['Calculated Concentration [ng/g]'])

print('Experiment 2')
print(f"t-test statistic: {stat}")
print(f"P-value: {p_value}")

Experiment 1
t-test statistic: 2.751009254545537
P-value: 0.040261943508780734
Experiment 2
t-test statistic: 10.29283872595399
P-value: 0.0005025325389158062


In [9]:
## with pingoiun gives more information

# Exp1
res = pg.ttest(df_1_front['Calculated Concentration µg/L'], df_1_cere['Calculated Concentration µg/L'], paired=True, correction=False)
print('Experiment 1')
display(res)

# Exp2
res = pg.ttest(df_2_front['Calculated Concentration [ng/g]'], df_2_cere['Calculated Concentration [ng/g]'], paired=True, correction=False)
print('Experiment 2')
display(res)

Experiment 1


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.751009,5,two-sided,0.040262,"[2.91, 85.93]",0.41302,2.447,0.131875


Experiment 2


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,10.292839,4,two-sided,0.000503,"[474.67, 825.34]",1.612996,62.384,0.76838
